In [1]:
# Importanción de librerías.
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import requests

In [3]:
# Credenciales de Spotify.
CLIENT_ID = 'a018711755a146b2a678a93741dc6041'
CLIENT_SECRET = '51fa44e4b57343f19c810b9a75089da6'

# Autenticación con Spotify.
auth_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(auth_manager=auth_manager)

In [4]:
# Búsqueda en la API de Spotify a través de la librería spotipy.
datos = sp.search(q="genre:rock year:2020", type='track', limit=50, offset=0)

In [5]:
# Géneros y rango de años elegidos.
genres = ["pop", "rock", "jazz", "hip hop"]
year_start = 2002
year_end = 2007

# Almacenar los resultados de canciones y álbumes en listas.
lista_canciones_albumes = [] 
lista_artistas = []

In [6]:
# Acceder a los detalles del álbum de la primera canción que aparece en la lista de resultados que te devuelve la API de Spotify.
datos['tracks']['items'][0]['album']['type']


'album'

In [ ]:
# Bucle para recorrer los géneros de la lista 'genres'.
for genre in genres: 
    # Realizar la búsqueda en la API de Spotify, buscando canciones (tracks) y álbumes (albums).
    # Se busca por género y por rango de años, limitando los resultados a 50 y estableciendo un desplazamiento (offset) de 951.
    datos = sp.search(q=f"genre:{genre} year:{year_start}-{year_end}", type='track,album', limit=50, offset=951)

    # Recorremos la lista de canciones encontradas en los resultados de la búsqueda.
    for item in datos['tracks']['items']: # 'tracks' contiene las canciones encontradas.
        
        # Agregamos, para cada canción, un diccionario con detalles relevantes a la lista.
        lista_canciones_albumes.append(
            {
                "nombre_cancion":item['name'],
                "nombre_artista":item['artists'][0]['name'],
                'genre':genre,                 
                "type":  item['album']['type'],
                "year": item['album']['release_date'][:4],
                "id": item['id']
            }
        )

        # Agregamos el nombre del artista a la lista.
        lista_artistas.append(
            {
                "nombre_artista":item['artists'][0]['name'],
            }
        )

In [38]:
# Longitud de la lista de artistas.
len(lista_artistas)

3850

In [39]:
# Longitud de la lista de canciones_albumes.
len(lista_canciones_albumes)

3850

In [40]:
# Convierte la lista de diccionarios en un DataFrame de Pandas.
df_lista_artistas = pd.DataFrame(lista_artistas)

In [41]:
# Limpia la lista lista_artistas. Si un artista aparece más de una vez en la columna, ahora solo aparecerá una vez en el conjunto.
lista_artistas_limpia = set(df_lista_artistas["nombre_artista"])


In [43]:
# Convierte la lista de diccionarios en un DataFrame de Pandas.
df_lista_canciones_albumes = pd.DataFrame(lista_canciones_albumes)

# Guarda el DataFrame de canciones en un archivo CSV, eliminando el índice, para que los datos puedan ser utilizados más fácilmente fuera de Python.
df_lista_canciones_albumes.to_csv("canciones.csv", index= False)


In [44]:
# Convierte la lista de diccionarios en un DataFrame de Pandas.
df_artistas_limpia = pd.DataFrame(lista_artistas_limpia)

# Guarda el DataFrame de artistas en un archivo CSV, eliminando el índice, para que los datos puedan ser utilizados más fácilmente fuera de Python.
df_artistas_limpia.to_csv("artistas.csv", index = False)

In [45]:
# Como ya tenemos los dos archivos CSV con la información de artistas, canciones y álbumes, desde la API de last.fm sacamos la información de cada artista.

# Credenciales de last.fm.
API_KEY = 'bb92344bdb091e7c718014c1dcd0e28d'  
BASE_URL = 'http://ws.audioscrobbler.com/2.0/'

In [48]:
# Declaración nueva lista para posteriormente añadir os datos
datos_artistas = []

In [49]:
# Bucle para recorrer la lista de artistas ya limpia.
for artista in lista_artistas_limpia:

    # Parámetros para hacer una solicitud a la API.
    params_info = {
        "method": "artist.getInfo", 
        "api_key": API_KEY,
        "format": "json", 
        "artist": artista
    }
    try:
        # Consulta de la API. Realizamos la solicitud GET a la API con los parámetros definidos.
        response_info = requests.get(BASE_URL, params = params_info)

        # Verificamos si la solicitud fue exitosa (código de estado 200), sino se imprime un mensaje de error con el nombre del artista y el código de estado.
        if response_info.status_code !=200:
            print(f"Error en la solicitud para {artista}: {response_info.status_code}")
            continue

        # Si la solicitud fue exitosa, convertimos la respuesta en formato JSON.
        data_json = response_info.json()

        # Extraemos la información del artista del JSON de la respuesta.
        artist_data = data_json.get('artist', {})

        # Extraemos el resumen biográfico del artista. Si no está disponible, se asigna "Información no disponible".
        resumen = artist_data.get('bio', {}).get('summary', "Información no disponible")

        # Extraemos la popularidad (número de oyentes). Si no está disponible, se asigna "No disponible".
        popularidad = artist_data.get('stats', {}).get('listeners', "No disponible")

        # Extraemos el número de reproducciones (playcount). Si no está disponible, se asigna "No disponible".
        reproducciones = artist_data.get('stats', {}).get('playcount', "No disponible")

        # Extraemos los artistas similares. Si no hay artistas similares, se asigna una lista vacía.
        similares = [sim['name'] for sim in artist_data.get('similar', {}).get('artist', [])]

        # Añadimos un diccionario con la información del artista a la lista declarada anteriormente.
        # El diccionario contiene el nombre del artista, el resumen biográfico, la popularidad, el número de reproducciones y los artistas similares.
        datos_artistas.append({"artista": artista, 
                               "resumen": resumen,
                                "popularidad": popularidad,
                                "reproducciones": reproducciones,
                                "similares": similares})
    
    # Si ocurre un error debido a que falta alguna clave en el JSON capturamos el KeyError y mostramos un mensaje indicando que falta información para ese artista.
    except KeyError:
        print(f"Falta información para el artista: {artista}")

    # Si ocurre cualquier otro error inesperado (por ejemplo, problemas de red o conexión con la API), capturamos la excepción general y mostramos el mensaje de error junto con el nombre del artista.
    except Exception as e:
        print(f"Ocurrió un error inesperado con {artista}: {e}")
        
    

Ocurrió un error inesperado con Luis Salinas: Expecting value: line 1 column 1 (char 0)


In [50]:
# Convierte la lista de diccionarios en un DataFrame de Pandas.
df_datos_artistas = pd.DataFrame(datos_artistas)

# Guarda el DataFrame de datos_artistas en un archivo CSV, eliminando el índice, para que los datos puedan ser utilizados más fácilmente fuera de Python.
df_datos_artistas.to_csv("datos_artistas.csv", index = False)